In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv 
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
path        = 'Traces_Cassandra'
fields      = ['W_95']
X_raw       = pd.read_csv(path+'/X_new_10mbit.csv')
y_raw       = pd.read_csv(path+'/Y_new_10mbit.csv', skipinitialspace=True, usecols=fields)


FileNotFoundError: [Errno 2] File b'Traces_Cassandra/X_new_10mbit.csv' does not exist: b'Traces_Cassandra/X_new_10mbit.csv'

In [7]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X_raw, y_raw['W_95'], test_size=0.3, random_state=11)
for t in ts:
    selectedMetrics = []
    bestMetrics = []
    possibleMetrics = [x for x in list(range(1, len(list(X_train))))] #gera indices das metricas no vetor list(X_train)
    lenPossibleMetrics = len(possibleMetrics)
    minNMAEGlobal = 10000.00
    yTestMean = y_test.mean()
    regr = RandomForestRegressor(max_depth=10, n_estimators=5, bootstrap=False)
    count = 0
    stop = False
    while(not stop):
        actualBestMetric = -1
        minNMAELocal = 10000.00
        for k in possibleMetrics:
            indexes = selectedMetrics + [k] #inclui a metrica de indice 'k' no conjunto de metricas a serem consideradas
            X_train_new = X_train.iloc[:, indexes] #filtra o conjunto de treino considerando apenas os indices selecionados
            X_test_new = X_test.iloc[:, indexes] #filtra o conjunto de teste considerando apenas os indices selecionados
            regr.fit(X_train_new, y_train) #monta o modelo de regressao
            pred = regr.predict(X_test_new) #executa estimativa com base no modelo criado
            nmae_Selected = (abs(pred - y_test).mean())/yTestMean #calcula o NMAE para o conjunto selecionado
            
            if nmae_Selected < minNMAELocal: #se houver melhora no NMAE calculado para essa rodada faz as devidas substituicoes
                minNMAELocal = nmae_Selected
                actualBestMetric = k
                
        selectedMetrics = selectedMetrics + [actualBestMetric]

        if minNMAELocal - minNMAEGlobal >= 0.000001: # se o minimo encontrado for pelo menos 0.000001 menor
            count = count + 1                        # que o minimo tido ate o momento, faz a substituicao
            if count == 10:
                stop = True # se 10 metricas testadas aumentam o menor erro obtido, interrompe-se o algoritmo
        else:               # do contrario, faz as devidas substituicoes e zera o contador  
            minNMAEGlobal = minNMAELocal
            bestMetrics = selectedMetrics
            count = 0
            
        possibleMetrics.pop(actualBestMetric) #retira a metrica selecionada do conjunto de metricas ainda disponiveis
        
        lenPossibleMetrics = lenPossibleMetrics - 1
        
        if lenPossibleMetrics <= 0:
            stop = True # se nao houver mais metricas a serem testadas, interrompe-se o algoritmo
    metricsName = [X_train_list[x] for x in bestMetrics]
    print("Menor erro atingido: ", minNMAEGlobal)
    print("Melhor K = ", len(bestMetrics))
    print("Metricas: ", metricsName)
    print("Erros obtidos: ", errors)
    print("O processo levou: ", (edGlobal - bgGlobal), " segundos.")

KeyboardInterrupt: 